In [2]:
import numpy
import numba

In [14]:
a = numpy.random.uniform(-2, 2, size=1000000).reshape(10000,100)
weights = numpy.exp(-numpy.linspace(-3, 3, 100)**2)
weights /= weights.sum()
weights[:10]

array([  4.21986594e-06,   6.04823582e-06,   8.60534661e-06,
         1.21539546e-05,   1.70402689e-05,   2.37161871e-05,
         3.27659589e-05,   4.49376634e-05,   6.11797478e-05,
         8.26826654e-05])

In [21]:
def np_weighted_avg(x, weights):
    return (x * weights).sum(axis=1)

In [39]:
np_time = %timeit -o np_weighted_avg(a, weights)

100 loops, best of 3: 4.03 ms per loop


In [31]:
@numba.jit
def np_compiled_weighted_avg(x, weights):
    return (x * weights).sum(axis=1)

In [24]:
%timeit nb_weighted_avg(a, weights)

The slowest run took 20.54 times longer than the fastest. This could mean that an intermediate result is being cached.
100 loops, best of 3: 3.68 ms per loop


In [36]:
@numba.jit(nopython=True)
def nb_weighted_avg_loops(x, weights):
    result = numpy.empty(x.shape[0], x.dtype)
    for i in range(x.shape[0]):

        row_sum = 0.0
        for v, w in zip(x[i], weights):
            row_sum += v*w
            
        result[i] = row_sum
        
    return result

In [37]:
numpy.testing.assert_allclose(np_weighted_avg(a, weights), nb_weighted_avg_loops(a, weights))

In [40]:
nb_loops_time = %timeit -o nb_weighted_avg_loops(a, weights)

1000 loops, best of 3: 1.53 ms per loop


In [41]:
np_time.best / nb_loops_time.best

2.6322916063187707

In [57]:
@numba.guvectorize(['(float64[:], float64[:], float64[:])', '(float32[:], float32[:], float32[:])'],
                   '(i),(i)->()')
def gufunc_weighted_avg(row, weights, result):
    row_sum = 0.0
    for v, w in zip(row, weights):
        row_sum += v*w
    result[0] = row_sum

In [63]:
gufunc_time = %timeit -o gufunc_weighted_avg(a, weights)

The slowest run took 7.88 times longer than the fastest. This could mean that an intermediate result is being cached.
1000 loops, best of 3: 985 µs per loop


In [59]:
numpy.testing.assert_allclose(np_weighted_avg(a, weights), gufunc_weighted_avg(a, weights))

In [64]:
np_time.best / gufunc_time.best

4.088664738999888